# **Анализ репозиториев в GitHub**

## 1. Введение
### 1.1 Цель исследования
Собрать данные о популярных GitHub-репозиториях по тегам AI и ML с помощью веб-парсинга результатов глобального поиска GitHub, провести очистку данных, создание новых признаков и исследовательский анализ, чтобы выявить закономерности популярности open-source проектов.

### 1.2 Источник данных

Мы выполняем парсинг HTML-страниц глобального поиска GitHub:

Глобальный поиск по тегу AI:
https://github.com/search?q=AI&type=repositories&s=stars&o=desc

Глобальный поиск по тегу ML:
https://github.com/search?q=ML&type=repositories&s=stars&o=desc

### 1.3 Описание предметной области

GitHub — крупнейшая платформа для open-source разработки.
Мы анализируем репозитории, отсортированные по популярности (метрика: количество звёзд, сортировка Most stars → по убыванию).
Это позволяет выявлять факторы, связанные с востребованностью проектов в области AI/ML.

---
### 1.4 Требования и результат

| Требование                             | Результат |
|:--------------------------------------:|:-----------------------------:|
| Количество наблюдений  > 1000          | 2000+                         |
| Начальное количество признаков >= 8-10 | 10                            |
| Типы признаков                         |                               |
| Количество новых признаков >= 2        |                               |
| Финальное количество признаков         |                               |
| Визуализации >= 3                      |                               |
| Итоговый формат                        | Jupyter/Colab notebook + .CSV |
| [дополнительно] Репозиторий на github  | ссылка                        |


### 1.5 Технологии проекта

| Технология          | Документация                                       |
| :------------------ | :------------------------------------------------: |
| Python 3.12.12      | https://docs.python.org/3/                         |
| pandas              | https://pandas.pydata.org/docs/                    |
| NumPy               | https://numpy.org/doc/                             |
| requests            | https://requests.readthedocs.io/                   |
| BeautifulSoup (bs4) | https://beautiful-soup-4.readthedocs.io/en/latest/ |
| time (stdlib)       | https://docs.python.org/3/library/time.html        |
| random (stdlib)     | https://docs.python.org/3/library/random.html      |
| itertools (stdlib)  | https://docs.python.org/3/library/itertools.html   |
| pathlib (stdlib)    | https://docs.python.org/3/library/pathlib.html     |
| types.NoneType      | https://docs.python.org/3/library/types.html       |


### 1.6 Команда и распределение задач

| Участник                         | Обязанности                         |
|----------------------------------|-------------------------------------|
| **Трофимов Матвей Владимирович** | Сбор, парсинг, первичные выводы     |
| **Самунджян Дина Арменаковна**   | Очистка данных, пропуски, дубликаты |
| **Серенко Елена Валерьевна**     | Создание новых признаков            |
| **Пащенко Дмитрий Игоревич**     | Визуализации, выводы и  результаты  |


### Структура ноутбука

```markdown
# 1. Введение
## 1.1 Цель проекта
## 1.2 Источник данных
## 1.3 Описание предметной области
## 1.4 Требования к датасету
## 1.5 Технологии проекта
## 1.6 Команда и распределение задач

Трофимов Матвей Владимирович  
### 2. Этап 0 — Сбор и парсинг данных
2.1 получение данных через парсинг
2.2 формирование датасета и сохранение `название.csv`
2.3 краткий просмотр результата и промежуточные выводы

Самунджян Дина Арменаковна  
### 3. Этап 1 — Очистка данных
3.1 обработка пропусков, дубликатов, выбросов
3.2 приведение типов данных
3.3 сохранение `название.csv`
3.4 краткие промежуточные выводы

Серенко Елена Валерьевна  
### 4. Этап 2 — Создание новых признаков
4.1 создание дополнительных признаков (не менее двух)
4.2 формирование `название.csv`
4.3 краткие промежуточные выводы

Пащенко Дмитрий Игоревич  
### 5. Этап 3 — Анализ данных (EDA) и визуализация
5.1 описательная статистика, группировки, корреляции
5.2 визуализации (не менее трёх разных типов)
5.3 интерпретация результатов и промежуточные выводы

### 6. Итоговые выводы исследования (все участники)
```

# Парсинг репозиториев с GitHub

# МАТВЕЙ

In [ ]:
"""
Этот код монтирует удаленную папку на диске с доступом для всех.
Далее все аутпуты файлов делайте в эту папку /content/drive/MyDrive/Colab Notebooks, чтобы у всех был доступ.
"""

from google.colab import drive
drive.mount('/content/drive')  # один раз на сеанс запустить, подтвердить доступ
shared_dir = '/content/drive/MyDrive/Colab Notebooks'  # путь к общей папке

Mounted at /content/drive


In [ ]:
import sys
import requests
import time
import pandas as pd
import random
import bs4
from bs4 import BeautifulSoup
from types import NoneType
import itertools
from pathlib import Path
import numpy as np

print("Python:", sys.version.split()[0])
print("requests:", requests.__version__)
print("pandas:", pd.__version__)
print("bs4:", bs4.__version__)
print("numpy:", np.__version__)

Python: 3.12.12
requests: 2.32.4
pandas: 2.2.2
bs4: 4.13.5
numpy: 2.0.2


### В рамках парсинга были созданы две вспомогательные функции:

**`repository_parsing(url)`**  
Функция принимает URL репозитория GitHub и с помощью HTTP‑запроса и библиотеки BeautifulSoup извлекает его параметры: название, описание, языки программирования, количество форков, звезд, коммитов, пул‑реквестов, контрибьюторов и релизов. При ошибке парсинга или отсутствии нужных элементов возвращает `None`, чтобы проблемные репозитории можно было безопасно пропустить при массовом сборе данных.

**`get_df_from_github(repositories_count)`**  
Функция выполняет массовый сбор данных о репозиториях по запросу `AI` и `ML` в поиске GitHub, обходя до 100 страниц выдачи с паузами между запросами. Для каждой найденной ссылки вызывается `repository_parsing`, а собранные параметры добавляются в списки до достижения заданного количества репозиториев `repositories_count`, после чего формируется `pandas.DataFrame` с основными метриками и ссылками на репозитории.

In [ ]:
# Получаем на вход ссылку на репозиторий в GitHub, парсим его и возвращаем лист с его параметрами
def repository_parsing(url):
    try:
        print(f'Парсим репозиторий: {url}')
        site1 = requests.get(url)
        site_text = site1.content.decode('utf-8')
        soup = BeautifulSoup(site_text, 'html.parser')
        split_url = url.split('/')
        repository_name = split_url[-2] + ' ' + split_url[-1]
        contributors = 0 # кол-во контрибьюторов репозитория
        releases = 0 # кол-во релизов репозитория
        for a in soup.find_all('a', class_='Link--primary no-underline Link'): # Находим информацию о кол-ве релизов репозитория
            splitted = a.text.split()
            if splitted[0] == 'Releases' and len(splitted) > 1:
                releases = int(splitted[1].replace(',', '').replace("+", ""))

        for a in soup.find_all('a', class_='Link--primary no-underline Link d-flex flex-items-center'): # Находим информацию о кол-ве котрибьюторов репозитория
            splitted = a.text.split()
            if splitted[0] == 'Contributors' and len(splitted) > 1:
                contributors = int(splitted[1].replace(',', '').replace("+", ""))

        languages = [] # список языков программирования, которые использовались в данном репозитории
        for span in soup.find_all('span', 'color-fg-default text-bold mr-1'):
            languages.append(span.text)

        pull_requests = 0 # кол-во пул реквестов в репозитории
        fork_amount = 0 # кол-во форков в репозитории
        stars_amount = 0 # кол-во звезд репозитория
        commits_amount = 0 # кол-во коммитов в репозитории

        if soup.find('p', class_='f4 my-3') != NoneType: # проверка наличия элемента на странице
            repository_description = soup.find('p', class_='f4 my-3').text.strip()

        if soup.find('span', class_='Counter', id='pull-requests-repo-tab-count') != NoneType: # проверка наличия элемента на странице
            pull_requests = int(soup.find('span', class_='Counter', id='pull-requests-repo-tab-count').get('title').replace(',', ''))

        if soup.find('span', id="repo-network-counter") != NoneType: # проверка наличия элемента на странице
            fork_amount = int(soup.find('span', id="repo-network-counter").get('title').replace(',', ''))

        if soup.find('span', id="repo-stars-counter-star") != NoneType: # проверка наличия элемента на странице
            stars_amount = int(soup.find('span', id="repo-stars-counter-star").get('title').replace(',', ''))

        if soup.find('span', class_='fgColor-default') != NoneType: # проверка наличия элемента на странице
            commits_amount = int(soup.find('span', class_='fgColor-default').text.split()[0].replace(',', ''))
        # возвращаем список с параметрами репозитория
        return [repository_name, repository_description, languages, fork_amount, stars_amount, commits_amount, pull_requests, contributors, releases, url]

    except Exception as e: # обработка ошибок при парсинге репозитория
        print(f"Ошибка при парсинге {url}: {e}")
        return None

In [ ]:
def get_df_from_github(repositories_count, query_tag):
    names = [] # список для названий репозиториев
    descriptions = [] # список для описаний репозиториев
    languages = [] # список языков программирования, используемых в репозитории
    fork_amounts = [] # список кол-ва форков в репозитории
    stars_amounts = [] # список кол-ва звезд репозитория
    commits_amounts = [] # список кол-ва коммитов в репозитории
    pull_requests_amounts = [] # список кол-ва пул-реквестов в репозитории
    contributors_amounts = [] # список кол-ва контрибьюторов репозитория
    releases_amounts = [] # список кол-ва релизов в репозитории
    urls = [] # список с ссылками на репозитории
    source_path = 'https://github.com'
    cur_repositories_count = 0
    site1 = requests.get(f'https://github.com/search?q={query_tag}&type=repositories&s=stars&o=desc')
    pages_count = repositories_count // 10 if repositories_count % 10 == 0 else repositories_count // 10 + 1
    i = 0
    while i < pages_count: # проверка, что номер страницы не больше 100, пагинация больше 100 страниц недоступна
      i += 1
      if (i != 1):
        time.sleep(10) # иммитация работы человека, пауза в работе кода на 10 сек
      print(f'Парсим репозитории с данного url: https://github.com/search?q=AI&type=repositories&s=stars&o=desc&p={i}')
      site1 = requests.get(f'https://github.com/search?q=AI&type=repositories&s=stars&o=desc&p={i}')
      print(f'Страница получена, с кодом ответа: {site1.status_code}')
      while (site1.status_code != 200): # продолжаем запрос страницы пока она не будет возвращена со статусом 200
        time.sleep(10)
        site1 = requests.get(f'https://github.com/search?q=AI&type=repositories&s=stars&o=desc&p={i}')
        print(f'Страница получена, с кодом ответа: {site1.status_code}')
      site_text = site1.content.decode('utf-8')
      soup = BeautifulSoup(site_text, 'html.parser')
      for link in soup.find_all('a', class_='Link__StyledLink-sc-1syctfj-0 prc-Link-Link-85e08'):
          if cur_repositories_count == repositories_count: # выход из цикла при достаточном кол-ве обработанных репозиториев
              break
          repository_stat = repository_parsing(source_path + link.get('href'))
          if repository_stat == None: # если обработать репозиторий не удалось переходим к следующему
            continue
          cur_repositories_count += 1
          # сохраняем полученную информацию
          names.append(repository_stat[0])
          descriptions.append(repository_stat[1])
          languages.append(repository_stat[2])
          fork_amounts.append(repository_stat[3])
          stars_amounts.append(repository_stat[4])
          commits_amounts.append(repository_stat[5])
          pull_requests_amounts.append(repository_stat[6])
          contributors_amounts.append(repository_stat[7])
          releases_amounts.append(repository_stat[8])
          urls.append(repository_stat[9])
    df = pd.DataFrame({
        'Name': names,
        'Description': descriptions,
        'Programming languages': languages,
        'Forks count': fork_amounts,
        'Stars count': stars_amounts,
        'Commits count': commits_amounts,
        'Pull requests count': pull_requests_amounts,
        'Contributors count': contributors_amounts,
        'Releases counts': releases_amounts,
        'Url': urls
    })
    return df

In [ ]:
#df1 = get_df_from_github(1000) # передаем кол-во репозиториев, которое хотим спарсить
df2 = get_df_from_github(25, 'AI')



Парсим репозитории с данного url: https://github.com/search?q=AI&type=repositories&s=stars&o=desc&p=1
Страница получена, с кодом ответа: 200
Парсим репозиторий: https://github.com/Significant-Gravitas/AutoGPT
Парсим репозиторий: https://github.com/n8n-io/n8n
Парсим репозиторий: https://github.com/AUTOMATIC1111/stable-diffusion-webui
Парсим репозиторий: https://github.com/langflow-ai/langflow
Парсим репозиторий: https://github.com/langchain-ai/langchain
Парсим репозиторий: https://github.com/langgenius/dify
Парсим репозиторий: https://github.com/open-webui/open-webui
Парсим репозиторий: https://github.com/microsoft/generative-ai-for-beginners
Парсим репозиторий: https://github.com/x1xhlol/system-prompts-and-models-of-ai-tools
Парсим репозиторий: https://github.com/comfyanonymous/ComfyUI
Парсим репозитории с данного url: https://github.com/search?q=AI&type=repositories&s=stars&o=desc&p=2
Страница получена, с кодом ответа: 200
Парсим репозиторий: https://github.com/supabase/supabase
Парси

In [ ]:
# df1.to_csv('github_repositories.csv', index=False) # сохраняем локально
df1.to_csv(f'/content/drive/MyDrive/Colab Notebooks/github_repositories1.csv', index=False) # сохраняем в общую папку в google drive
df2.to_csv(f'/content/drive/MyDrive/Colab Notebooks/github_repositories2.csv', index=False) # сохраняем в общую папку в google drive

NameError: name 'df1' is not defined

# ДИНА

### Общий обзор данных

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/github_repositories.csv")

print("Shape:", df.shape)
df.head()

Shape: (992, 10)


,Name,Description,Programming languages,Forks count,Stars count,Commits count,Pull requests count,Contributors count,Releases counts,Url
0,Significant-Gravitas AutoGPT,AutoGPT is the vision of accessible AI for eve...,"['Python', 'TypeScript', 'Dart', 'JavaScript',...",46180,180076,7606,86,783,83,https://github.com/Significant-Gravitas/AutoGPT
1,n8n-io n8n,Fair-code workflow automation platform with na...,"['TypeScript', 'Vue', 'SCSS', 'Python', 'JavaS...",51326,160338,16635,657,567,446,https://github.com/n8n-io/n8n
2,AUTOMATIC1111 stable-diffusion-webui,Stable Diffusion web UI,"['Python', 'JavaScript', 'CSS', 'HTML', 'Other']",29467,158707,7689,65,586,27,https://github.com/AUTOMATIC1111/stable-diffus...
3,langflow-ai langflow,Langflow is a powerful tool for building and d...,"['Python', 'TypeScript', 'JavaScript', 'CSS', ...",8108,140120,16936,569,328,273,https://github.com/langflow-ai/langflow
4,langchain-ai langchain,🦜🔗 The platform for reliable agents.,"['Python', 'Jupyter Notebook', 'Makefile', 'Sh...",19957,121028,14834,113,3812,1096,https://github.com/langchain-ai/langchain


### Обзор структуры

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 992 entries, 0 to 991
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Name                   992 non-null    object
 1   Description            992 non-null    object
 2   Programming languages  992 non-null    object
 3   Forks count            992 non-null    int64 
 4   Stars count            992 non-null    int64 
 5   Commits count          992 non-null    int64 
 6   Pull requests count    992 non-null    int64 
 7   Contributors count     992 non-null    int64 
 8   Releases counts        992 non-null    int64 
 9   Url                    992 non-null    object
dtypes: int64(6), object(4)
memory usage: 77.6+ KB


In [ ]:
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Name,992,992,NevaMind-AI memU,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Description,992,991,The Cloud-Native API Gateway and AI Gateway,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Programming languages,992,599,[],85,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Forks count,992.0,NaN,NaN,NaN,1915.578629,4269.563053,0.0,465.0,806.5,1813.75,60727.0
Stars count,992.0,NaN,NaN,NaN,12901.775202,17872.077446,2919.0,4192.25,6698.0,13528.25,180076.0
Commits count,992.0,NaN,NaN,NaN,2881.295363,8737.465559,1.0,221.75,747.0,2356.25,200324.0
Pull requests count,992.0,NaN,NaN,NaN,32.466734,77.413491,0.0,2.0,9.0,27.0,700.0
Contributors count,992.0,NaN,NaN,NaN,94.944556,232.673728,0.0,10.0,30.0,83.0,3812.0
Releases counts,992.0,NaN,NaN,NaN,77.173387,282.389473,0.0,0.0,10.0,63.0,5000.0
Url,992,992,https://github.com/NevaMind-AI/memU,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Обработка пропусков

In [ ]:
obj_cols = df.select_dtypes(include=["object"]).columns
df[obj_cols] = df[obj_cols].replace(r"^\s*$", np.nan, regex=True)

Пустые строки в текстовых (object) колонках приведены к NaN, чтобы пропуски обрабатывались единообразно.

In [ ]:
if "Description" in df.columns:
    df["Description"] = df["Description"].fillna("")

Description: пропуски заполнены пустой строкой "" (чтобы анализ текста не ломался).

In [ ]:
if "Programming languages" in df.columns:
    langs_str = df["Programming languages"].astype(str).str.strip()
    df["languages_missing"] = df["Programming languages"].isna() | langs_str.eq("[]")
    df.loc[df["languages_missing"], "Programming languages"] = "Unknown"

Отметили строки, где языки не указаны (NaN или "[]"), создали для них флаг languages_missing и заменили значение в Programming languages на "Unknown", чтобы дальше удобно анализировать.

In [ ]:
key_cols = [c for c in ["Name", "Url"] if c in df.columns]
before = len(df)
df = df.dropna(subset=key_cols)
print("Удалено строк из-за пропусков в ключевых полях:", before - len(df))

Удалено строк из-за пропусков в ключевых полях: 0


In [ ]:
print("\nПропуски по колонкам после очистки:\n", df.isna().sum().sort_values(ascending=False))


Пропуски по колонкам после очистки:
 Name                     0
Description              0
Programming languages    0
Forks count              0
Stars count              0
Commits count            0
Pull requests count      0
Contributors count       0
Releases counts          0
Url                      0
languages_missing        0
dtype: int64


### Обработка дубликатов

In [ ]:
if "Url" in df.columns:
    df = df.drop_duplicates(subset=["Url"], keep="first")

df = df.drop_duplicates()

print("Удалено дубликатов:", before - len(df))

Удалено дубликатов: 0


Удалили строки-дубликаты (сначала по Url, затем полностью совпадающие), чтобы каждый репозиторий учитывался один раз и метрики не дублировались.

### Очистка от выбросов

In [ ]:
k = 1.5
need_at_least = 3  # удаляем, если выбросов >= 3 метрик

releases_col = "Releases count" if "Releases count" in df.columns else "Releases counts"
count_cols = [c for c in ["Forks count","Stars count","Commits count","Pull requests count","Contributors count",releases_col] if c in df.columns]

flags = pd.DataFrame(index=df.index)

for col in count_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    right = Q3 + k * IQR
    flags[col] = df[col] > right

to_drop = flags.sum(axis=1) >= need_at_least

before = len(df)
df = df[~to_drop].copy()
print("Удалено строк (выбросы по >=3 метрикам):", before - len(df))

Удалено строк (выбросы по >=3 метрикам): 91


Для выбросов использовали множественный критерий: строка удалялась только если превышала порог IQR (Q3+1.5⋅IQR) сразу по нескольким (≥3) метрикам. Такой подход выбран как более мягкий, поскольку GitHub-показатели имеют выраженный “длинный хвост”, и одиночные экстремальные значения часто соответствуют реально популярным репозиториям, поэтому множественный критерий снижает риск удаления валидных наблюдений.

# ЛЕНА

https://docs.google.com/document/d/19Wz_Qp2rlGZ58ArQdpFhYuXK9widnxpfIVhatFABlOQ/edit?tab=t.0

Лена и черновик от ГПТ

# ДИМА